In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

In [2]:
class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

In [3]:
def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

In [4]:
def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}

In [5]:
workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

checkpointer = InMemorySaver()

graph = workflow.compile(checkpointer=checkpointer)

In [6]:
config: RunnableConfig = {"configurable": {"thread_id": "1"}}

graph.invoke({"foo": "", "bar":[]}, config)

{'foo': 'b', 'bar': ['a', 'b']}

In [7]:
# get the latest state snapshot
config = {"configurable": {"thread_id": "1"}}
graph.get_state(config)

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0ff697-bbf2-62fa-8002-a108aa572e2f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-01T12:28:16.125782+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0ff697-bbe9-6fa8-8001-8fb85c3b3d36'}}, tasks=(), interrupts=())

In [8]:
# get a state snapshot for a specific checkpoint_id
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f0ff697-bbe9-6fa8-8001-8fb85c3b3d36"}}
graph.get_state(config)

StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0ff697-bbe9-6fa8-8001-8fb85c3b3d36'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-02-01T12:28:16.122420+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0ff697-bbe1-6f09-8000-45d47432b022'}}, tasks=(PregelTask(id='b3f47d13-dad9-4ad3-1446-91944f5f9b4a', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts=(), state=None, result={'foo': 'b', 'bar': ['b']}),), interrupts=())

In [9]:
config = {"configurable": {"thread_id": "1"}}
list(graph.get_state_history(config))

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0ff697-bbf2-62fa-8002-a108aa572e2f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-01T12:28:16.125782+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0ff697-bbe9-6fa8-8001-8fb85c3b3d36'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0ff697-bbe9-6fa8-8001-8fb85c3b3d36'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-02-01T12:28:16.122420+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0ff697-bbe1-6f09-8000-45d47432b022'}}, tasks=(PregelTask(id='b3f47d13-dad9-4ad3-1446-91944f5f9b4a', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts